In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv


In [2]:
df = pd.read_csv("../input/sentiment140/training.1600000.processed.noemoticon.csv",encoding='latin-1')
df.columns=['score','id','datetime','flag','user','text']
df

,score,id,datetime,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [3]:
!pip install ekphrasis
from ekphrasis.classes.segmenter import Segmenter

     |████████████████████████████████| 80 kB 341 kB/s            
  Preparing metadata (setup.py) ... - done
     |████████████████████████████████| 64 kB 869 kB/s            
  Preparing metadata (setup.py) ... - done
  Created wheel for ekphrasis: filename=ekphrasis-0.5.1-py3-none-any.whl size=82842 sha256=4324dabbd9b108184f3b1ea2f318dcbd7bda001b125292f54d014c70f1d4a2e1
  Stored in directory: /root/.cache/pip/wheels/f7/ec/0d/12659e32faf780546945d0120f2c8410eb3efb7426731da88f
  Created wheel for ftfy: filename=ftfy-6.0.3-py3-none-any.whl size=41933 sha256=4933652f18f4c863a7736085d3e443c5565ad057d6941769f6fd0b7ff62cc9bf
  Stored in directory: /root/.cache/pip/wheels/19/f5/38/273eb3b5e76dfd850619312f693716ac4518b498f5ffb6f56d
Successfully built ekphrasis ftfy


In [4]:
import regex as re
#important libraries for preprocessing using NLTK
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
import numpy as np

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
df.isna().sum()

score       0
id          0
datetime    0
flag        0
user        0
text        0
dtype: int64

In [6]:
df['hashtag']=df['text'].apply(lambda x:re.findall(r"#(\w+)", x))

In [7]:
df=df.drop(df.columns[[0, 1, 2, 3, 4]], axis = 1)
df

,text,hashtag
0,is upset that he can't update his Facebook by ...,[]
1,@Kenichan I dived many times for the ball. Man...,[]
2,my whole body feels itchy and like its on fire,[]
3,"@nationwideclass no, it's not behaving at all....",[]
4,@Kwesidei not the whole crew,[]
...,...,...
1599994,Just woke up. Having no school is the best fee...,[]
1599995,TheWDB.com - Very cool to hear old Walt interv...,[]
1599996,Are you ready for your MoJo Makeover? Ask me f...,[]
1599997,Happy 38th Birthday to my boo of alll time!!! ...,[]


In [8]:
# segmenter using the word statistics from Twitter
seg_tw = Segmenter(corpus="twitter")
a = []
for i in range(len(df)):
 if df['hashtag'][i] != a:
  listToStr1 = ' '.join([str(elem) for elem in df['hashtag'][i]])
  df.loc[i,'Segmented#'] = seg_tw.segment(listToStr1)

Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt


/opt/conda/lib/python3.7/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [9]:
df['Segmented#']=df['Segmented#'].fillna('')
df['text']=df['text'].astype(str)+df['Segmented#']
df=df.drop(df.columns[[1, 2]], axis = 1)
df

,text
0,is upset that he can't update his Facebook by ...
1,@Kenichan I dived many times for the ball. Man...
2,my whole body feels itchy and like its on fire
3,"@nationwideclass no, it's not behaving at all...."
4,@Kwesidei not the whole crew
...,...
1599994,Just woke up. Having no school is the best fee...
1599995,TheWDB.com - Very cool to hear old Walt interv...
1599996,Are you ready for your MoJo Makeover? Ask me f...
1599997,Happy 38th Birthday to my boo of alll time!!! ...


In [10]:
def preprocess_data(data):
 #Removes Numbers
 data = data.astype(str).str.replace('\d+', '')
 lemmatizer = nltk.stem.WordNetLemmatizer()
 w_tokenizer =  TweetTokenizer()
 stop_words = set(stopwords.words('english'))
 def lemmatize_text(text):
  return [(lemmatizer.lemmatize(w)) for w in w_tokenizer.tokenize((text))]
 def remove_punctuation(words):
  new_words = []
  finalwords = []
  for word in words:
      new_word = re.sub(r'[^\w\s]', '', (word))
      if new_word!='':
         new_words.append(new_word)
  for word in new_words:
    if len(word)>2:
        finalwords.append(word)
  return finalwords
 words = data.apply(lemmatize_text)
 words = words.apply(remove_punctuation)
 words = words.apply(lambda x: [item for item in x if item not in stop_words])
 return pd.DataFrame(words)

In [11]:
dfnew=np.array_split(df, 10)
dfnew[0]

,text
0,is upset that he can't update his Facebook by ...
1,@Kenichan I dived many times for the ball. Man...
2,my whole body feels itchy and like its on fire
3,"@nationwideclass no, it's not behaving at all...."
4,@Kwesidei not the whole crew
...,...
159995,@KimokoMasada I wished I could tell you to sto...
159996,in London at the mo. It's ridiculously hot and...
159997,Just one more day in San Jose and then we're o...
159998,@WerewolfEmbry She *is* kind of right though E...


In [12]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

for i in range(10):
    dfnew[i]['text']=dfnew[i]['text'].apply(lambda x:re.sub(r'@\w+', '', x).strip())
    dfnew[i]['text']=dfnew[i]['text'].apply(lambda x: emoji_pattern.sub(r'', x).lower())
    dfnew[i]['text']=dfnew[i]['text'].apply(lambda x:re.sub(r'#\w+', '', x).strip())

In [13]:
dfnew

[                                                     text
 0       is upset that he can't update his facebook by ...
 1       i dived many times for the ball. managed to sa...
 2          my whole body feels itchy and like its on fire
 3       no, it's not behaving at all. i'm mad. why am ...
 4                                      not the whole crew
 ...                                                   ...
 159995  i wished i could tell you to stop tweeting 'ca...
 159996  in london at the mo. it's ridiculously hot and...
 159997  just one more day in san jose and then we're o...
 159998  she *is* kind of right though embers...sometim...
 159999  i know  i was listenin to bad habit earlier an...
 
 [160000 rows x 1 columns],
                                                      text
 160000  layin n bed with a headache  ughhhh...waitin o...
 160001                funeral ceremony...gloomy friday...
 160002               wants to hang out with friends soon!
 160003  we want to trade 

In [14]:
for i in range(10):
    dfnew[i]['text']=preprocess_data(dfnew[i]['text'])

dfnew

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


[                                                     text
 0       [upset, cant, update, facebook, texting, might...
 1       [dived, many, time, ball, managed, save, rest,...
 2                  [whole, body, feel, itchy, like, fire]
 3                              [behaving, mad, cant, see]
 4                                           [whole, crew]
 ...                                                   ...
 159995  [wished, could, tell, stop, tweeting, cause, b...
 159996  [london, ridiculously, hot, left, short, home,...
 159997  [one, day, san, jose, officially, vacation, di...
 159998  [kind, right, though, ember, sometimes, butthead]
 159999  [know, listenin, bad, habit, earlier, started,...
 
 [160000 rows x 1 columns],
                                                      text
 160000       [layin, bed, headache, ughhhh, waitin, call]
 160001                [funeral, ceremony, gloomy, friday]
 160002                         [want, hang, friend, soon]
 160003       [want, trade

In [15]:
for i in range(10):
    x=dfnew[i]
    dfnew[i]=x[x['text'].map(len)> 2]
dfnew

[                                                     text
 0       [upset, cant, update, facebook, texting, might...
 1       [dived, many, time, ball, managed, save, rest,...
 2                  [whole, body, feel, itchy, like, fire]
 3                              [behaving, mad, cant, see]
 6       [hey, long, time, see, yes, rain, bit, bit, lo...
 ...                                                   ...
 159995  [wished, could, tell, stop, tweeting, cause, b...
 159996  [london, ridiculously, hot, left, short, home,...
 159997  [one, day, san, jose, officially, vacation, di...
 159998  [kind, right, though, ember, sometimes, butthead]
 159999  [know, listenin, bad, habit, earlier, started,...
 
 [142512 rows x 1 columns],
                                                      text
 160000       [layin, bed, headache, ughhhh, waitin, call]
 160001                [funeral, ceremony, gloomy, friday]
 160002                         [want, hang, friend, soon]
 160003       [want, trade

In [16]:
df=pd.concat(dfnew)
df=df.reset_index(drop=True)
df

,text
0,"[upset, cant, update, facebook, texting, might..."
1,"[dived, many, time, ball, managed, save, rest,..."
2,"[whole, body, feel, itchy, like, fire]"
3,"[behaving, mad, cant, see]"
4,"[hey, long, time, see, yes, rain, bit, bit, lo..."
...,...
1407545,"[woke, school, best, feeling, ever]"
1407546,"[thewdbcom, cool, hear, old, walt, interview, ..."
1407547,"[ready, mojo, makeover, ask, detail]"
1407548,"[happy, birthday, boo, alll, time, tupac, amar..."


In [17]:
df.to_csv('sentiment140dataset.csv')